In [1]:
# warnin 메시지 표시안함
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd

In [2]:
nsmc_train_df = pd.read_csv("data/ratings_train.txt",encoding='utf8', sep="\t", engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
nsmc_train_df=nsmc_train_df[nsmc_train_df['document'].notnull()]
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [5]:
nsmc_train_df['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [6]:
import re
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅎ|가-힣]+'," ", x ))
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
nsmc_test_df=pd.read_csv('data/ratings_test.txt', encoding='utf8', sep="\t", engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [8]:
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [9]:
nsmc_test_df=nsmc_test_df[nsmc_test_df['document'].notnull()]

In [10]:
nsmc_test_df['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [11]:
nsmc_test_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅎ|가-힣]+'," ", x ))
nsmc_test_df.head()

,id,document,label
0,6270596,아 더빙 진짜 짜증나네요 목소리,1
1,9274899,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,0
2,8544678,너무재밓었다그래서보는것을추천한다,0
3,6825595,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6723715,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,0


In [12]:
!pip install konlpy

In [13]:
from konlpy.tag import Okt  # 형태소 분석
okt = Okt()

In [14]:
def okt_tokenizer(text):    # ㅎ여태소 분석 - > 형태소 단위로 토큰화
    tokens = okt.morphs(text)
    return tokens

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9, token_pattern=None)

In [17]:
tfidf.fit(nsmc_train_df['document'])

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2), token_pattern=None,
                tokenizer=<function okt_tokenizer at 0x00000260CBF2CE00>)

In [18]:
nsmc_train_tfidf =  tfidf.transform(nsmc_train_df['document'])

In [19]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state=0,max_iter=500)
SA_lr.fit(nsmc_train_tfidf,nsmc_train_df['label'])

LogisticRegression(max_iter=500, random_state=0)

In [20]:
from sklearn.model_selection import GridSearchCV

params={'C':[1,3,3.5,4,4.5,5]} #최적화 하이퍼 매개변수 후보군
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params,cv=3,scoring='accuracy',verbose=1)
#최적화 하이퍼 파라미터 찾아서 정확도가 제일 높은 최적 모델로 생성
SA_lr_grid_cv.fit(nsmc_train_tfidf,nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=500, random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [21]:
print(SA_lr_grid_cv.best_params_ , round(SA_lr_grid_cv.best_score_, 4 ))
SA_lr_best = SA_lr_grid_cv.best_estimator_

{'C': 4.5} 0.8545


In [24]:

nsmc_test_tfidf = tfidf.transform(nsmc_test_df['document'])
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:

from sklearn.metrics import accuracy_score
print(f'감성 분석 정확도 :' , round(accuracy_score(nsmc_test_df['label'],test_predict),3))


In [ ]:
st = "웃자 ^o^ 오늘은 좋은 날이 될 것 같은 예감100%! ^^*"
st= re.compile(r'[ㄱ-ㅎ|가-힣]+').findall(st)
print(st)
st=[" ".join(st)]
print(st)

In [ ]:

#1) 입력텍스트 피처 벡터화
st_tfidf = tfidf.transform(st)
#2) 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = SA_lr_best.predict(st_tfidf)
#3) 예측값 출력하기
if(st_predict==0):
    print(st,"->> 부정 감성")
else :
    print(st,"--> 긍정 감성")